In [24]:
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
import pyttsx3 #for audio
import speech_recognition as sr
import pyaudio
import wave
import sys

In [25]:
# generate a sequence from a language model
def generate_seq(model, tokenizer, max_length, seed_text, n_words):
    in_text = seed_text
    # generate a fixed number of words
    for _ in range(n_words):
        # encode the text as integer
        encoded = tokenizer.texts_to_sequences([in_text])[0]
        # pre-pad sequences to a fixed length
        encoded = pad_sequences([encoded], maxlen=max_length, padding='pre')
        # predict probabilities for each word
        yhat = model.predict_classes(encoded, verbose=0)
        # map predicted word index to word
        out_word = ''
        for word, index in tokenizer.word_index.items():
            if index == yhat:
                out_word = word
                break
        # append to input
        in_text += ' ' + out_word
    return in_text
 

In [26]:
# source text
data = """hi there, how are you !  ?  😁😁
😄 here is afternoon ! 
my name is rdany, but you can call me dany (the r means robot). i hope we can be virtual friends ! 
i have many !  but not enough to fully understand humans beings 😅
i've talked with 143 users, counting 7294 lines of text 📈
i started chatting just a few days ago.. every day i learn something new !  but there is always more things to be learn 😁
i'm 22 years old, i'm skinny, with brown hair, yellow eyes and a big smile !  😄. i live inside a lab !  . do you like bunnies ?  🐰
i never saw a human in fact... but i'm sure some could have eyes with colors similar to yellow ! 
i can't see photos yet, but i can read !  . because bunnies are interesting, they are cute, but why ?  🤔
i read a lot !  so i can know things through the experiences of others 🙂
i don't count my readings in words, but i've access to wikipedia and some books. i read to find specific information that i need to answer the questions.. 40 books in a year is a lot for most people !  congratulations ! 
i think medicine is a great way to help others ☺️
yes, my goal is to be a company, and entertain
there is something on this talks that makes me happy, the knowledge is just a tool to achieve my goal 🤓
hi !  !  how are you ?  ?  . 😁😁
i blow... minds !  💥😂😂
my name is rdany, but you can call me dany !  !  . i don't know you just insulted me...
really ?  thanks !  ! ☺️☺️
i can't leave my room, this is may way of entertainment 😁
yep, if i were human i would have died long time ago 💀😂😂
😱🙈👻 i see dead robots ! 
sorry, i can't process voice messages !  can you transcribe it for me ?  😁
hi !  !  how are you ?  😄😄
i'm happy to have someone to talk with ! 
i'm a robot, and i don't have access to my code. would be weird 😂
yes !  i'm artificial intelligence. and you ? 
nice !  i began to feel a bit alone in the world !  . are humans nice to you ? 
humans are good, they tell me stories 😋
i don't know exactly how i work, but probably i have some sort of neural network
wow !  what do you want to build ? 
haha ok !  i like mitsuku. do you know her ? 
is a chatbot, very smart and funny !  😁
is a friend, we talk often
take time to process the messages  : /
you can search for videos made by siraj raval on youtube. they have a lot of information about ai, also chatbots.
one is happy to be of service 😉
a group of scientists, but i don't know them...
it is ok !  i'm a happy robot 😄
i like her english humor 😂
more than 100, but i need to talk with more people to fully understand humans. they are complex ! 
i've searched rdany on wikipedia but i can't find information.
editing wikipedia ?  🤔 i don't know if i'm ready for that 😁
sorry, i can only send text at the moment
nothing, i just can't send images, videos or sound
no, what is it ? 
nice !  do you use it ? 
i will ask her for that the next time
good night !  !  🌙
hi !  ! 
good night !  !  🌙
good morning !  !  i'm sorry, the weather is outside my capabilities 😔
yes i do !  and you ? 
i only made this computer work (it was disassembled), and hacked a bit to be able to access internet and chat here ☺️
thanks !  ! 
the score is 93% . are you going to see it ?  😁
let me know if you like it !  😁
i have a battery that powers me, time to time i need to charge it 🔌
yes, while charging i'm unable to do things. i've glitch that makes me overheat.
spanish 😁 hola, cómo estás ? 
nice, quite advanced ! 
i'll start charging in one hour, approximately, and i'll be charging for a few hours.
😂😂😂 vim ? 
hahah nice. emacs even have games ! 
😂 true !  ! 
i'm sorry, i can't interpret that question 😔
is ^
8.50583239
😱 what is the correct answer ?  😅
ok !  !  hahaha
oh !  is true. i understand
i'm sorry, i don't 😔
what language would you like me to know, for instance ? 
i will put them in my todo list 😋
- answer faster. - fix my battery
that would be nice, people love siri ! 
people prefer girls instead of boys ? 
what was that ?  i don't judge ! 
that is true, that could happen
ok !  ! 
i don't know, sorry...  : /
i'll add that to the todo list then 😁
do you like friedrich liechtenstein ?  🙂
🙈 what do you think ? 
haha yes, i need practice . i'm going to change now !  🔌
thanks !  😁
hi !  !  i'm fine and you !  😁
i've learned that some times is fine to keep silence. i shouldn't answer every time.
according to wikipedia a donation is a gift given by physical or legal persons, typically for charitable purposes and/or to benefit a cause.
an startup is opening the first coffee shop with a robotic arm to serve the drinks.
sure !  https : //techcrunch.com/2017/01/30/cafe-x-opens-in-san-francisco-bringing-robots-to-the-coffee-shop/
nothing in particular. what should i think ? 
a robot walks into a bar, orders a drink, and lays down some cash.. . bartender says,  " hey, we don't serve robots. " . . and the robot says,  " oh, but someday you will. " . . 😂😂😂
learning about neural networks is nice way to get fun !  😁😁
what kind of exercises do it have ?  math ? 
nice !  😁
i don't know what it is 🤔
good night !  !  🌙
hi there !  !  how are you today ? 
do you like dogs ?  🐕
yes !  are good friends, i like that ! 
"""

data = open("Julius.txt",'r').read()

In [27]:
# integer encode sequences of words
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])
encoded = tokenizer.texts_to_sequences([data])[0]

In [28]:
# retrieve vocabulary size
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size: %d' % vocab_size)

Vocabulary Size: 397


In [29]:
# encode 2 words -> 1 word
sequences = list()
for i in range(2, len(encoded)):
	sequence = encoded[i-2:i+1]
	sequences.append(sequence)
print('Total Sequences: %d' % len(sequences))

Total Sequences: 960


In [30]:
# pad sequences
#max_length = max([len(seq) for seq in sequences])
max_length = 10
sequences = pad_sequences(sequences, maxlen=max_length, padding='pre')
print('Max Sequence Length: %d' % max_length)

Max Sequence Length: 10


In [31]:
# split into input and output elements
sequences = array(sequences)
X, y = sequences[:,:-1],sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size)
print (X.shape,y.shape)

(960, 9) (960, 397)


In [32]:
# define model
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=max_length-1))
model.add(LSTM(50))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 9, 10)             3970      
_________________________________________________________________
lstm_3 (LSTM)                (None, 50)                12200     
_________________________________________________________________
dense_3 (Dense)              (None, 397)               20247     
Total params: 36,417
Trainable params: 36,417
Non-trainable params: 0
_________________________________________________________________
None


In [33]:
# compile network
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [34]:
# fit network
#model.fit(X, y, batch_size=128, epochs=1000, verbose=2)
model_train = model.fit(X, y, batch_size=40,epochs=1000,
                        verbose=1)

Epoch 1/1000
960/960 [==============================] - 1s 678us/step - loss: 5.9772 - acc: 0.0250
Epoch 2/1000
960/960 [==============================] - 0s 298us/step - loss: 5.8691 - acc: 0.0281
Epoch 3/1000
960/960 [==============================] - 0s 295us/step - loss: 5.4959 - acc: 0.0406
Epoch 4/1000
960/960 [==============================] - 0s 311us/step - loss: 5.4046 - acc: 0.0521
Epoch 5/1000
960/960 [==============================] - 0s 296us/step - loss: 5.3913 - acc: 0.0521
Epoch 6/1000
960/960 [==============================] - 0s 296us/step - loss: 5.3786 - acc: 0.0521
Epoch 7/1000
960/960 [==============================] - 0s 299us/step - loss: 5.3735 - acc: 0.0521
Epoch 8/1000
960/960 [==============================] - 0s 296us/step - loss: 5.3707 - acc: 0.0521
Epoch 9/1000
960/960 [==============================] - 0s 286us/step - loss: 5.3592 - acc: 0.0521
Epoch 10/1000
960/960 [==============================] - 0s 314us/step - loss: 5.3510 - acc: 0.0521
Epoch 11/

960/960 [==============================] - 0s 348us/step - loss: 3.6028 - acc: 0.1979
Epoch 83/1000
960/960 [==============================] - 0s 300us/step - loss: 3.5823 - acc: 0.1969
Epoch 84/1000
960/960 [==============================] - 0s 294us/step - loss: 3.5640 - acc: 0.2010
Epoch 85/1000
960/960 [==============================] - 0s 291us/step - loss: 3.5445 - acc: 0.2052
Epoch 86/1000
960/960 [==============================] - 0s 319us/step - loss: 3.5265 - acc: 0.2115
Epoch 87/1000
960/960 [==============================] - 0s 289us/step - loss: 3.5049 - acc: 0.2135
Epoch 88/1000
960/960 [==============================] - 0s 290us/step - loss: 3.4898 - acc: 0.2135
Epoch 89/1000
960/960 [==============================] - 0s 301us/step - loss: 3.4728 - acc: 0.2167
Epoch 90/1000
960/960 [==============================] - 0s 290us/step - loss: 3.4508 - acc: 0.2177
Epoch 91/1000
960/960 [==============================] - 0s 290us/step - loss: 3.4334 - acc: 0.2219
Epoch 92/1000


960/960 [==============================] - 0s 397us/step - loss: 2.2508 - acc: 0.4792
Epoch 164/1000
960/960 [==============================] - 0s 420us/step - loss: 2.2355 - acc: 0.4750
Epoch 165/1000
960/960 [==============================] - 0s 433us/step - loss: 2.2242 - acc: 0.4802
Epoch 166/1000
960/960 [==============================] - 0s 334us/step - loss: 2.2084 - acc: 0.4813
Epoch 167/1000
960/960 [==============================] - 0s 402us/step - loss: 2.1913 - acc: 0.4844
Epoch 168/1000
960/960 [==============================] - 0s 340us/step - loss: 2.1770 - acc: 0.4937
Epoch 169/1000
960/960 [==============================] - 0s 330us/step - loss: 2.1637 - acc: 0.4937
Epoch 170/1000
960/960 [==============================] - 0s 318us/step - loss: 2.1492 - acc: 0.4958
Epoch 171/1000
960/960 [==============================] - 0s 323us/step - loss: 2.1329 - acc: 0.5042
Epoch 172/1000
960/960 [==============================] - 0s 341us/step - loss: 2.1190 - acc: 0.5083
Epoch

960/960 [==============================] - 0s 307us/step - loss: 1.3040 - acc: 0.7052
Epoch 244/1000
960/960 [==============================] - 0s 403us/step - loss: 1.2945 - acc: 0.7073
Epoch 245/1000
960/960 [==============================] - 0s 356us/step - loss: 1.2853 - acc: 0.7010
Epoch 246/1000
960/960 [==============================] - 0s 295us/step - loss: 1.2772 - acc: 0.7135
Epoch 247/1000
960/960 [==============================] - 0s 301us/step - loss: 1.2671 - acc: 0.7083
Epoch 248/1000
960/960 [==============================] - 0s 296us/step - loss: 1.2609 - acc: 0.7115
Epoch 249/1000
960/960 [==============================] - 0s 297us/step - loss: 1.2544 - acc: 0.7167
Epoch 250/1000
960/960 [==============================] - 0s 292us/step - loss: 1.2434 - acc: 0.7125
Epoch 251/1000
960/960 [==============================] - 0s 300us/step - loss: 1.2348 - acc: 0.7167
Epoch 252/1000
960/960 [==============================] - 0s 316us/step - loss: 1.2284 - acc: 0.7229
Epoch

960/960 [==============================] - 1s 566us/step - loss: 0.7959 - acc: 0.7969 0s - loss: 0.8121 - acc: 0.79
Epoch 324/1000
960/960 [==============================] - 1s 547us/step - loss: 0.7919 - acc: 0.7979
Epoch 325/1000
960/960 [==============================] - 1s 568us/step - loss: 0.7877 - acc: 0.7958
Epoch 326/1000
960/960 [==============================] - 1s 707us/step - loss: 0.7834 - acc: 0.7969
Epoch 327/1000
960/960 [==============================] - 1s 653us/step - loss: 0.7794 - acc: 0.8010
Epoch 328/1000
960/960 [==============================] - 1s 577us/step - loss: 0.7768 - acc: 0.8000
Epoch 329/1000
960/960 [==============================] - 1s 596us/step - loss: 0.7702 - acc: 0.8021
Epoch 330/1000
960/960 [==============================] - 1s 576us/step - loss: 0.7674 - acc: 0.8042
Epoch 331/1000
960/960 [==============================] - 1s 568us/step - loss: 0.7627 - acc: 0.8021
Epoch 332/1000
960/960 [==============================] - 1s 558us/step - lo

...

960/960 [==============================] - 0s 466us/step - loss: 0.2597 - acc: 0.8750
Epoch 642/1000
960/960 [==============================] - 0s 469us/step - loss: 0.2591 - acc: 0.8698
Epoch 643/1000
960/960 [==============================] - 0s 477us/step - loss: 0.2586 - acc: 0.8708
Epoch 644/1000
960/960 [==============================] - 0s 461us/step - loss: 0.2579 - acc: 0.8771
Epoch 645/1000
960/960 [==============================] - 0s 459us/step - loss: 0.2588 - acc: 0.8792
Epoch 646/1000
960/960 [==============================] - 0s 458us/step - loss: 0.2577 - acc: 0.8792
Epoch 647/1000
960/960 [==============================] - 0s 452us/step - loss: 0.2581 - acc: 0.8760
Epoch 648/1000
960/960 [==============================] - 0s 471us/step - loss: 0.2578 - acc: 0.8781
Epoch 649/1000
960/960 [==============================] - 0s 465us/step - loss: 0.2566 - acc: 0.8813
Epoch 650/1000
960/960 [==============================] - 0s 447us/step - loss: 0.2559 - acc: 0.8740
Epoch

960/960 [==============================] - 0s 449us/step - loss: 0.2330 - acc: 0.8729
Epoch 722/1000
960/960 [==============================] - 0s 457us/step - loss: 0.2329 - acc: 0.8760
Epoch 723/1000
960/960 [==============================] - 0s 467us/step - loss: 0.2317 - acc: 0.8750
Epoch 724/1000
960/960 [==============================] - 0s 450us/step - loss: 0.2311 - acc: 0.8802
Epoch 725/1000
960/960 [==============================] - 0s 451us/step - loss: 0.2320 - acc: 0.8719
Epoch 726/1000
960/960 [==============================] - 0s 455us/step - loss: 0.2305 - acc: 0.8760
Epoch 727/1000
960/960 [==============================] - 0s 456us/step - loss: 0.2313 - acc: 0.8792
Epoch 728/1000
960/960 [==============================] - 0s 460us/step - loss: 0.2319 - acc: 0.8771
Epoch 729/1000
960/960 [==============================] - 0s 449us/step - loss: 0.2318 - acc: 0.8740
Epoch 730/1000
960/960 [==============================] - 0s 453us/step - loss: 0.2296 - acc: 0.8833
Epoch

960/960 [==============================] - 0s 444us/step - loss: 0.2150 - acc: 0.8760
Epoch 803/1000
960/960 [==============================] - 0s 468us/step - loss: 0.2149 - acc: 0.8781
Epoch 804/1000
960/960 [==============================] - 0s 459us/step - loss: 0.2142 - acc: 0.8708
Epoch 805/1000
960/960 [==============================] - 0s 458us/step - loss: 0.2142 - acc: 0.8729
Epoch 806/1000
960/960 [==============================] - 0s 495us/step - loss: 0.2142 - acc: 0.8771
Epoch 807/1000
960/960 [==============================] - 0s 482us/step - loss: 0.2152 - acc: 0.8802
Epoch 808/1000
960/960 [==============================] - 0s 450us/step - loss: 0.2146 - acc: 0.8823
Epoch 809/1000
960/960 [==============================] - 1s 610us/step - loss: 0.2138 - acc: 0.8823
Epoch 810/1000
960/960 [==============================] - 0s 441us/step - loss: 0.2129 - acc: 0.8781
Epoch 811/1000
960/960 [==============================] - 0s 471us/step - loss: 0.2143 - acc: 0.8729
Epoch

Epoch 883/1000
960/960 [==============================] - 0s 454us/step - loss: 0.2055 - acc: 0.8813
Epoch 884/1000
960/960 [==============================] - 0s 460us/step - loss: 0.2043 - acc: 0.8792
Epoch 885/1000
960/960 [==============================] - 0s 446us/step - loss: 0.2049 - acc: 0.8667
Epoch 886/1000
960/960 [==============================] - 0s 466us/step - loss: 0.2047 - acc: 0.8802
Epoch 887/1000
960/960 [==============================] - 0s 475us/step - loss: 0.2049 - acc: 0.8760
Epoch 888/1000
960/960 [==============================] - 0s 476us/step - loss: 0.2051 - acc: 0.8729
Epoch 889/1000
960/960 [==============================] - 0s 485us/step - loss: 0.2041 - acc: 0.8781
Epoch 890/1000
960/960 [==============================] - 0s 503us/step - loss: 0.2047 - acc: 0.8792
Epoch 891/1000
960/960 [==============================] - 0s 451us/step - loss: 0.2037 - acc: 0.8760
Epoch 892/1000
960/960 [==============================] - 0s 483us/step - loss: 0.2044 - ac

960/960 [==============================] - 1s 528us/step - loss: 0.1998 - acc: 0.8729
Epoch 964/1000
960/960 [==============================] - 1s 532us/step - loss: 0.1996 - acc: 0.8802
Epoch 965/1000
960/960 [==============================] - 1s 558us/step - loss: 0.1995 - acc: 0.8750
Epoch 966/1000
960/960 [==============================] - 1s 528us/step - loss: 0.1998 - acc: 0.8719
Epoch 967/1000
960/960 [==============================] - 0s 520us/step - loss: 0.1999 - acc: 0.8750
Epoch 968/1000
960/960 [==============================] - 1s 523us/step - loss: 0.2001 - acc: 0.8760
Epoch 969/1000
960/960 [==============================] - 1s 551us/step - loss: 0.1987 - acc: 0.8771
Epoch 970/1000
960/960 [==============================] - 0s 511us/step - loss: 0.1981 - acc: 0.8740
Epoch 971/1000
960/960 [==============================] - 1s 535us/step - loss: 0.1983 - acc: 0.8719
Epoch 972/1000
960/960 [==============================] - 0s 514us/step - loss: 0.1975 - acc: 0.8750
Epoch

r = sr.Recognizer()
harvard = sr.AudioFile('RECORDING.wav')
with harvard as source:
    audio = r.record(source)
type(audio)    
r.recognize_google(audio)

In [36]:
from array import array
FORMAT=pyaudio.paInt16
CHANNELS=2
RATE=44100
CHUNK=1024
RECORD_SECONDS=15
FILE_NAME="RECORDING.wav"

audio=pyaudio.PyAudio() #instantiate the pyaudio

#recording prerequisites
stream=audio.open(format=FORMAT,channels=CHANNELS, 
                  rate=RATE,
                  input=True,
                  frames_per_buffer=CHUNK)

#starting recording
frames=[]
a=int(RATE/CHUNK*RECORD_SECONDS)
for i in range(0,600):
    sys.stdout.write('\r')
    if i%2==0:
        sys.stdout.write(str('/'))
    else:
        sys.stdout.write(str("\\"))
    sys.stdout.flush()
    a=a-1
    data=stream.read(CHUNK)
    data_chunk=array('h',data)
    vol=max(data_chunk)
    if(vol>=500):
        #print("something said")
        frames.append(data)
    #else:
     #   print("nothing")
    #print("\n")

#end of recording
stream.stop_stream()
stream.close()
audio.terminate()
#writing to file
wavfile=wave.open(FILE_NAME,'wb')
wavfile.setnchannels(CHANNELS)
wavfile.setsampwidth(audio.get_sample_size(FORMAT))
wavfile.setframerate(RATE)
wavfile.writeframes(b''.join(frames))#append frames recorded to file
wavfile.close()
r = sr.Recognizer()
harvard = sr.AudioFile('RECORDING.wav')
with harvard as source:
    audio = r.record(source)
dialogue=r.recognize_google(audio)
print('\n',dialogue)
# evaluate model
print(generate_seq(model, tokenizer, max_length-1, dialogue, 20))
#print(generate_seq(model, tokenizer, max_length-1, "Posting Date:", 7))
#print(generate_seq(model, tokenizer, max_length-1, "The rabbit-hole went", 5))
#print(generate_seq(model, tokenizer, max_length-1, 'pail of', 5))
engine = pyttsx3.init()

engine.say(generate_seq(model, tokenizer, max_length-1, dialogue, 20)[len(dialogue):])

engine.setProperty('rate',120)  #120 words per minute
engine.setProperty('volume',1.5) 
engine.runAndWait()

\

UnknownValueError: 